# Imports

In [ ]:
import cvxpy as cp
import numpy as np
import random
from scipy.optimize import linprog
from tqdm import tqdm
from time import time
import matplotlib.pyplot as plt

def printf(s):
  s = '\n\033[43m'+ '\033[1m' + str(s) + '\033[0m \n'
  print(s)

# Random Test case generator

In [ ]:
def create(m,n):

  if type(n) is int:
    n = [n]

  R = []; C = []
  for i in range(len(n)):
    R.append(np.random.randint(100,size=(m,n[i])))
    C.append(np.random.randint(100,size=(m,n[i])))
  
  p = [1/len(n)]*len(n)
  return R,C,p


# DOBSS and DOBSS modification 1

In [ ]:
# Stackelberg game solution -> without change
def formulate_sg(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [#z<=1,q<=1, #feasibility, but this is implied by the following constraints
         
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue

       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Stackelberg game solution 
def formulate_sg2(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [
         
        #  z<=1,q<=1, #feasibility, but this is implied by the following constraints
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

        #  x == sum(z.T), # by definition
        #  0<=a-x@C, a-x@C-1e3*(1-q)<=0, #follower's problem -> maximizing follower revenue
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue
         sum(sum(cp.multiply(R,z))) <= Δ + sum(z.T)@R + M*( sum(sum(cp.multiply(C,z))) - sum(z.T)@C ) # Select the payoff of leader which minimum among all available maximizing strategies of follower
       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Bayesian Stackelberg game solution 
def formulate_bsg(R,C,p,solver=1,Δ=0,lp=0):

  for l in range(len(R)):
    if solver==0:
      o,c,z,q = formulate_sg(R[l],C[l],p[l],Δ=Δ,lp=lp)
    elif solver==1:
      o,c,z,q = formulate_sg2(R[l],C[l],p[l],Δ=Δ,lp=lp)

    if l == 0:
      Obj = o
      Con = c
      Z = [z]
      Q = [q]
    else:
      Obj += o
      Con += c
      Z +=[z]
      Q +=[q]

  return Obj,Con,Z,Q 


# Solver of the game 
def optimal_strategy(R,C,p,show=False,solver=0,Δ=0,lp=0):

  obj, con, Z, Q = formulate_bsg(R,C,p,solver=solver,Δ=Δ,lp=lp)

  if len(Z)>1:
    c2 = []
    for l in range(len(Z)):
      if l==0:
        k = sum(Z[l].T)
      else:
        c2+=[k==sum(Z[l].T)] # additional constraint for all xi's to be equal    
    con+=c2

  prob = cp.Problem(obj, con)
  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  for i in range(len(Z)):
    if i == 0:
      zf = np.sum(np.asarray(Z[i].value),axis=1)/len(Z)
    else:
      zf += np.sum(np.asarray(Z[i].value),axis=1)/len(Z)


  ans = 0
  ans2 = 0
  for i in range(len(Z)):
    ans += np.sum(Z[i].value*R[i])*p[i]
    ans2 += np.sum(Z[i].value*C[i])*p[i]


  if show==True:

    print("-------------------------------------------------")
    printf('Solver='+str(solver)+'  Δ='+str(Δ))
    print("Optimal value: ",prob.value,'\n\n')
    print("Leader\'s strategy",'\n',zf,'\n\n')
    print("Follower strategies - for different types")
    for i in range(len(Z)):
      print("Player type",i," : ", Q[i].value)
    print('\n')
    print("-------------------------------------------------")

  return ans, ans2, Z # strategy profile


# DOBSS modification 2,3

In [ ]:
def formulate_sg_ir2(R,C,p,K=1/2,L=10,frac=0.10,solver=0,lp=0):

  m,n = R.shape
  # M = max(np.max(abs(R)),np.max(abs(C)))+1e3
  M=1e3

  K2=1

  if K<=1:
    K2=K
    K=1

  z = cp.Variable((m,n), nonneg=True)
  w = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  r = cp.Variable(n,integer=True)
  g = cp.Variable(n,integer=True) 
  a = cp.Variable(1)

  con = []

  if solver!=0:
    L = cp.Variable(1)
    con+=[L==a*frac]

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,w))))
  con +=[
         # Constraints from DOBSS
         sum(sum(z))==1,                                                                                     # Sum of all xi's=1     -> belongs to probability simplex
         sum(z.T)<=1,                                                                                        # Individual xi's <=1   -> belongs to probability simplex
         0<=q, sum(q)==1,                                                                                    # Sum of all qj's=1     -> belongs to probability simplex
         q<=sum(z), sum(z)<=1,                                                                               # As zij=xi.qj          -> definition of zij's
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0,                                                           #           a           -> Max follower payoff for given leader strategy

         # Constraints for modelling Irrational behaviour         
         g<=1, 0<=g,                                                                                         # Truth value variables -> binary
         sum(z.T) == sum(w.T),                                                                               # As wij=xi.q'j         -> definition of wij's
         r<=sum(w),
         0<=r, sum(r)==1,
         K2*(sum(sum(cp.multiply(C,w))) - sum(w.T)@C) >= (sum(sum(cp.multiply(R,w))) - sum(w.T)@R)/K - M*g,  # Comparisions          -> the loss of follower < k*(loss of leader) 
         a - sum(sum(cp.multiply(C,w))) <= L,                                                                # Stop loss condition   -> the loss of follower <= stop loss
         a - sum(w.T)@C >= L - M*(1-g),                                                                      # Stop loss criterion   -> truth value from g 
         a - sum(w.T)@C <= L + M*g                                                                           # Stop loss criterion   -> truth value from g 
       ]


  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]
    con+=[sum(w.T)==x]

  return obj, con, w, z, q, g, a, r



# Bayesian Stackelberg game solution 
def formulate_bsg2(R,C,p,solver=0,K=2,L=10,frac=0.1,lp=0):

  for l in range(len(R)):

    o, c, w, z, q, g, a, r = formulate_sg_ir2(R[l],C[l],p[l],K=K,L=L,solver=solver,frac=frac,lp=lp)    

    if l == 0:
      Obj = o
      Con = c
      W = [w]
      Z = [z]
      Q = [q]
      G = [g]
      A = [a]
      Rr = [r]

    else:
      Obj += o
      Con += c
      W += [w]
      Z += [z]
      Q += [q]
      G += [g]
      A += [a]
      Rr += [r]

  return Obj,Con,W,Z,Q,G,A,Rr


def optimal_strategy2(R,C,p,show=False,K=2,L=10,solver=0,frac=0,lp=0):

  Obj,Con,W,Z,Q,G,A,Rr = formulate_bsg2(R,C,p,K=K,L=L,solver=solver,frac=frac,lp=lp)

  if len(W)>1:
    c2 = []
    for l in range(len(W)):
      if l==0:
        k = sum(W[l].T)
      else:
        c2+=[k==sum(W[l].T)] # additional constraint for all xi's to be equal    
    Con+=c2

  prob = cp.Problem(Obj,Con)
  result = prob.solve()

  ans = 0
  ans2 = 0
  for i in range(len(W)):
    ans += np.sum(W[i].value*R[i])*p[i]
    ans2 += np.sum(W[i].value*C[i])*p[i]

  for i in range(len(W)):
    if i == 0:
      wf = np.sum(np.asarray(W[i].value),axis=1)
    else:
      wf += np.sum(np.asarray(W[i].value),axis=1)
  wf = wf/len(W)


  if show == True:

    if solver == 0:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Stop loss ='+str(L))
    else:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Fractional loss ='+str(frac))

    print("-------------------------------------------------")
    print("Objective value "+str(ans))

    print("-------------------------------------------------")
    print("W")
    for i in range(len(W)):
      print("player type ",i+1)
      print(W[i].value)

    print("-------------------------------------------------")
    print("Z")
    for i in range(len(Z)):
      print("player type ",i+1)
      print(Z[i].value)
    
    print("-------------------------------------------------")
    print("Q")
    for i in range(len(Q)):
      print("player type ",i+1)
      print(Q[i].value)

    print("-------------------------------------------------")
    print("G")
    for i in range(len(G)):
      print("player type ",i+1)
      print(G[i].value)
    
    print("-------------------------------------------------")
    print("A")
    for i in range(len(A)):
      print("player type ",i+1)
      print(A[i].value)

    print("-------------------------------------------------")
    print("Rr")
    for i in range(len(Rr)):
      print("player type ",i+1)
      print(Rr[i].value)

    print("-------------------------------------------------")

    print("Leader payoff: ",ans,'\n\n')
    
    print("Follower payoff - for different types")
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(W[i].value*C[i]))
    print('\n')

    print("-------------------------------------------------")

    print("Leader\'s strategy",'\n',wf,'\n\n')

    print("Follower strategies - for different types")
    L2 = []
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(np.asarray(W[i].value),axis=0))
      L2.append(np.sum(np.asarray(W[i].value),axis=0))
    print('\n')
    
    for i in range(len(W)):
      print("W matrix",i," : ", np.asarray(W[i].value))
    print('\n')

    print("-------------------------------------------------")


  return ans, ans2, W 
  

# Comparing solutions


# 3


Testing on different sizes of game matrices (which are randomly generated).
We observe that in the case of Modified DOBSS:-

* The payoff of the leader is **always** less than the payoff using original DOBSS and sometimes it is very significantly low, indicating the presence of atleast one of the following scenarios in the stackelberg setting:
  - **Weakly dominant follower strategy** which was missed 
  - **High risk strategies** which are very dependent on the rational behaviour of the follower, i.e. we neglect the irrational behaviour that 'if the follower wishes to take a loss, the leader would take comparitively, a much bigger loss'.
  - **Breaking ties against favour** of the leader

* But the payoff for the follower is somtimes greater or lesser than the payoff recieved using DOBSS solution (as DOBSS always gives the best possible solution), the former setting is not to be neglected because the follower is gaining because of his/her irrationality and it can be inferred that he/she posseses some **leverage** over the situation.

* Sometimes the solutions of the modifications and original Algorithm are exactly the same - indicating that there are none of the aforementioned anomalies are present.



In [ ]:
def small_game(R,C,W1):

  R1 = []
  C1 = []
  for i in range(len(R)):
    R1.append(   np.asarray([(R[i].T).dot( np.sum((W1[i]).value,axis=1))])  )
    C1.append(   np.asarray([(C[i].T).dot( np.sum((W1[i]).value,axis=1))])  )

  return R1,C1

In [ ]:
M = [2,3,4,5]
N = [[2,2,2,2], [3,3,3,3]] 
    #  , [4,4,4,4], [5,5,5,5]]


for i in range(len(M)):
  for j in range(len(N)):
    R,C,p = create(M[i],N[j])
    

    v1,f1,Z1 = optimal_strategy(R,C,p,show=False,solver=0)
    R1,C1 = small_game(R,C,Z1)
    v12,f12,Z12 = optimal_strategy(R1,C1,p,show=False,solver=1,Δ=0)
    v13,f13,Z13 = optimal_strategy2(R1,C1,p,show=False,K=2,L=10,solver=0)

    print("Original DOBSS output strategy evaluation")
    print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))
    print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v12,2),'|',round(f12,2)))
    print( "{:<15}".format("Mod-2 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v13,2),'|',round(f13,2)))

    print("---------------------------------------------")

    v2,f2,Z2 = optimal_strategy(R,C,p,show=False,solver=1,Δ=0)
    R2,C2 = small_game(R,C,Z2)
    v21,f21,Z21 = optimal_strategy(R2,C2,p,show=False,solver=0)
    v23,f23,Z23 = optimal_strategy2(R2,C2,p,show=False,K=2,L=10,solver=0)

    print("Modification-1 DOBSS output strategy evaluation - breaking ties against follower\'s wish")
    print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))
    print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v21,2),'|',round(f21,2)))
    print( "{:<15}".format("Mod-2 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v23,2),'|',round(f23,2)))

    print("---------------------------------------------")

    v3,f3,Z3 = optimal_strategy2(R,C,p,show=False,K=2,L=10,solver=0)
    R3,C3 = small_game(R,C,Z3)
    v31,f31,Z31 = optimal_strategy(R3,C3,p,show=False,solver=0)
    v32,f32,Z32 = optimal_strategy(R3,C3,p,show=False,solver=1,Δ=0)

    print("Modification-2 DOBSS output strategy evaluation - less risky strategy")
    print( "{:<15}".format("Mod-2 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
    print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v31,2),'|',round(f31,2)))
    print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v32,2),'|',round(f32,2)))

    print("---------------------------------------------")

    # # print(v1,'\n',v2,'\n',v3,'\n',v4,'\n')
    # # print(f1,'\n',f2,'\n',f3,'\n',f4,'\n')

    # print("Number of Leader strategies=",j)    
    # print("Number of Follower strategies=",j)    
    # print(  "{:<15}".format(" "),"{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
    # print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))
    # print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))
    # print( "{:<15}".format("Mod-2 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
    # print( "{:<15}".format("Mod-3 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v4,2),'|',round(f4,2)))

    # if v3>v1 or v3>v2:
    #   print(R,'\n',C)

    print("---------------------------------------------\n\n")
    

Original DOBSS output strategy evaluation
Original DOBSS  73.58      | 57.83     
Mod-1 DOBSS     67.14      | 57.83     
Mod-2 DOBSS     67.14      | 57.83     
---------------------------------------------
Modification-1 DOBSS output strategy evaluation - breaking ties against follower's wish
Mod-1 DOBSS     73.46      | 57.86     
Original DOBSS  73.46      | 57.86     
Mod-2 DOBSS     67.03      | 57.85     
---------------------------------------------
Modification-2 DOBSS output strategy evaluation - less risky strategy
Mod-2 DOBSS     71.25      | 57.0      
Original DOBSS  71.25      | 57.0      
Mod-1 DOBSS     71.25      | 57.0      
---------------------------------------------
---------------------------------------------


Original DOBSS output strategy evaluation
Original DOBSS  60.91      | 60.23     
Mod-1 DOBSS     55.61      | 60.23     
Mod-2 DOBSS     44.64      | 58.54     
---------------------------------------------
Modification-1 DOBSS output strategy evaluatio